<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/main/try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 12.6MB/s 
     |████████████████████████████████| 890kB 54.7MB/s 
     |████████████████████████████████| 3.2MB 43.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3d980ab6ccd2b3f66daa31fb22b962290082aaab768bca2b9cea0d413ec490b4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
#import torch

from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2DoubleHeadsModel.from_pretrained('gpt2')

# Add special tokens to vocabulary
spTokens = ["PICARD:", "DATA:", "TROI:", "LAFORGE:", "WORF:", "RIKER:", "CRUSHER:"]

num_added_tokens = tokenizer.add_special_tokens({"additional_special_tokens": spTokens})

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight', 'multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
embedding_layer = model.resize_token_embeddings(len(tokenizer))

sample = ["PICARD: We've no fear of what the true facts about us will reveal.", "DATA: It is possible, sir. But absolutely no margin for error.",
          "TROI: It it felt like something beyond what we'd consider a life form.", "LAFORGE: Which would affect how this works. No. Choice number two?",
          "WORF: Sir, sickbay reports Lieutenant Torres's condition is better.", "RIKER: Would I seem ungrateful if I ask for some information?",
          "CRUSHER: Wesley! I believe that means he would like us to be friends."]


In [5]:

encoded_samples = [tokenizer.encode(s) for s in sample]

#add_tokens_loc = [tokens.index(tokenizer.additional_special_tokens_ids) for tokens in encoded_samples]

add_tokens_loc = []

for token in encoded_samples:
  for spec_tok in tokenizer.additional_special_tokens_ids:
    if spec_tok in token:
      add_tokens_loc.append(token.index(spec_tok))

print(tokenizer.additional_special_tokens_ids)
print(encoded_samples)

[50257, 50258, 50259, 50260, 50261, 50262, 50263]
[[50257, 1135, 1053, 645, 3252, 286, 644, 262, 2081, 6419, 546, 514, 481, 7766, 13], [50258, 1026, 318, 1744, 11, 15967, 13, 887, 5543, 645, 10330, 329, 4049, 13], [50259, 1026, 340, 2936, 588, 1223, 3675, 644, 356, 1549, 2074, 257, 1204, 1296, 13], [50260, 13828, 561, 2689, 703, 428, 2499, 13, 1400, 13, 18502, 1271, 734, 30], [50261, 22788, 11, 6639, 24406, 3136, 21586, 27663, 338, 4006, 318, 1365, 13], [50262, 17353, 314, 1283, 555, 2164, 11850, 611, 314, 1265, 329, 617, 1321, 30], [50263, 54, 49048, 0, 314, 1975, 326, 1724, 339, 561, 588, 514, 284, 307, 2460, 13]]


In [ ]:
print(add_tokens_loc)

[0, 0, 0, 0, 0, 0, 0]


Train the new token embeddings

In [6]:
m = []
for x in encoded_samples:
  m.append(len(x))

print(m)

[15, 14, 15, 14, 13, 14, 16]


In [7]:
maxLen = max(m)
print(maxLen)

16


Padd to same lenght

In [8]:
index = 0

while index < len(encoded_samples):
  if (len(encoded_samples[index])) < maxLen:
    padding_length = maxLen - len(encoded_samples[index])
    print(padding_length)
    count = 0
    while count < padding_length:
      encoded_samples[index].append(0)
      count += 1
  index +=1   



print(encoded_samples)

1
2
1
2
3
2
[[50257, 1135, 1053, 645, 3252, 286, 644, 262, 2081, 6419, 546, 514, 481, 7766, 13, 0], [50258, 1026, 318, 1744, 11, 15967, 13, 887, 5543, 645, 10330, 329, 4049, 13, 0, 0], [50259, 1026, 340, 2936, 588, 1223, 3675, 644, 356, 1549, 2074, 257, 1204, 1296, 13, 0], [50260, 13828, 561, 2689, 703, 428, 2499, 13, 1400, 13, 18502, 1271, 734, 30, 0, 0], [50261, 22788, 11, 6639, 24406, 3136, 21586, 27663, 338, 4006, 318, 1365, 13, 0, 0, 0], [50262, 17353, 314, 1283, 555, 2164, 11850, 611, 314, 1265, 329, 617, 1321, 30, 0, 0], [50263, 54, 49048, 0, 314, 1975, 326, 1724, 339, 561, 588, 514, 284, 307, 2460, 13]]


check padding worked

In [9]:
m = []
for x in encoded_samples:
  m.append(len(x))

print(m)

[16, 16, 16, 16, 16, 16, 16]


Add attention mask

In [10]:
attention_mask = []
for x in encoded_samples:
    item = []
    for y in x:
      if y != 0:
        item.append(1)
      else:
        item.append(0)
    attention_mask.append(item)      
print(attention_mask)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [11]:
input_ids = torch.tensor(encoded_samples)
mc_token_ids = torch.tensor(add_tokens_loc)
mask = torch.tensor(attention_mask)

outputs = model(input_ids, mc_token_ids=mc_token_ids, attention_mask = mask)
lm_logits = outputs.logits
mc_logits = outputs.mc_logits

In [12]:
print(lm_logits)

tensor([[[ -37.0914,  -36.6030,  -40.4981,  ...,   -0.5582,    1.0085,
             1.2470],
         [-118.0642, -117.5912, -120.8277,  ...,   -4.2808,    2.5363,
             5.5928],
         [-137.3246, -137.2854, -141.4348,  ...,   -2.2893,    3.9849,
             6.0015],
         ...,
         [ -95.7057,  -96.4068, -104.3263,  ...,   -3.1154,    2.0381,
             4.2050],
         [-151.8244, -150.2855, -151.3961,  ...,   -4.9760,    3.0521,
             5.4891],
         [-126.6293, -124.9765, -126.1382,  ...,   -5.2103,    2.4705,
             4.7602]],

        [[ -37.8895,  -37.3718,  -41.2147,  ...,   -0.5485,    1.0319,
             1.2583],
         [ -78.2801,  -78.5836,  -80.8379,  ...,   -3.9691,    1.2780,
             3.2733],
         [-101.4379, -101.9793, -108.2304,  ...,   -3.1013,    2.7352,
             4.6446],
         ...,
         [-148.1863, -145.0646, -147.4446,  ...,   -5.0503,    2.8539,
             5.3013],
         [-135.2493, -132.8611, -136.007

Train on Textfile.

In [13]:
from google.colab import files
uploaded = files.upload()

Saving TNGScriptCut.json to TNGScriptCut.json


extract data from json

In [12]:
import json
import re
import numpy as np
from sklearn.model_selection import train_test_split

data = json.load(open('drive/MyDrive/Honours2021/TNGScriptCut2.json', 'r'))

In [13]:
script = []
episodes = data.keys()
part = "Scenes"

for ep in episodes:
  script.append(data[ep][part])

In [14]:
print(len(script))

def build_text_files(data_json, dest_path):
  f = open(dest_path, "w", encoding="utf-8")
  data = ""
  for block in data_json:
    for texts in block:
      summary = str(texts).strip()
      data += summary + " "
  print(str(len(data)))
  f.write(data)

train, test = train_test_split(script, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

176
4323337
791555
Train dataset length: 149
Test dataset length: 27


Training and datasetBuilder

In [16]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

from transformers import TextDataset, DataCollatorForLanguageModeling


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128)

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [23]:

from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,
    data_collator=data_collator,            # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

trainer.save_model("./TNG")

Step,Training Loss
500,2.955200


test

In [25]:

from transformers import pipeline

makeItSo = pipeline('text-generation',model='./TNG', tokenizer=tokenizer,config={'max_length':1200})

Some weights of the model checkpoint at ./TNG were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
makeItSo("Picard: make it so.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Picard: make it so.What is it? \nBATAI: This is the ship of the Doctor.We have discovered that the vessel is a prototype of the Klingon vessel known as T'Pol. \nBATA"}]

In [27]:
makeItSo("PICARD: Would you object to your captain")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'PICARD: Would you object to your captain making me watch that child? \nQUINN: I would not wish to see such a thing as an experiment.Captain, I am a man of integrity. \nQUINN: I have'}]

In [28]:
makeItSo("DATA: Can I ask you a question?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'DATA: Can I ask you a question? \nBARCLAY: Yes?Yes, Doctor. \nBARCLAY: Why would Doctor Beverly ask that? Why would you ask that.I just finished writing the thesis for'}]

In [29]:
makeItSo("Riker: Can I ask you a question Troi?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Riker: Can I ask you a question Troi?Is that Lieutenant Worf?  TROI : Commander Data, we're having difficulty accessing the sensors.Onscreen.It seems to be a malfunction.  REPU"}]